## The essential parts of the code comes from [this example notebook](https://drive.google.com/file/d/1XjFhd3eXx704tcdfQ4Q1OQn0LWKCRNJm/view)

In [1]:
import json, time, urllib.parse
import requests
import pandas as pd

df = pd.read_csv('thank_the_academy.AUG.2023b.csv.csv')

In [2]:
#########
#
#    CONSTANTS
#

# The REST API 'pageviews' URL - this is the common URL/endpoint for all 'pageviews' API requests
API_REQUEST_PAGEVIEWS_ENDPOINT = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/'

# This is a parameterized string that specifies what kind of pageviews request we are going to make
# In this case it will be a 'per-article' based request. The string is a format string so that we can
# replace each parameter with an appropriate value before making the request
API_REQUEST_PER_ARTICLE_PARAMS = 'per-article/{project}/{access}/{agent}/{article}/{granularity}/{start}/{end}'

# The Pageviews API asks that we not exceed 100 requests per second, we add a small delay to each request
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making a request to the Wikimedia API they ask that you include your email address which will allow them
# to contact you if something happens - such as - your code exceeding rate limits - or some other error 
REQUEST_HEADERS = {
    'User-Agent': '<hww1999@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2023',
}

# This is just a list of English Wikipedia article titles that we can use for example requests
# ARTICLE_TITLES = [ 'Bison', 'Northern flicker', 'Red squirrel', 'Chinook salmon', 'Horseshoe bat' ]
ARTICLE_TITLES = list(df['name'])

# This template is used to map parameter values into the API_REQUST_PER_ARTICLE_PARAMS portion of an API request. The dictionary has a
# field/key for each of the required parameters. In the example, below, we only vary the article name, so the majority of the fields
# can stay constant for each request. Of course, these values *could* be changed if necessary.
ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE = {
    "project":     "en.wikipedia.org",
    "access":      "desktop",      # this should be changed for the different access types. desktop,mobile-app,mobile-web
    "agent":       "user",
    "article":     "",             # this value will be set/changed before each request
    "granularity": "monthly",
    "start":       "2015070100",   # start and end dates need to be set, e.g. 2015010100
    "end":         "2023100100"    # this is likely the wrong end date, e.g. 2023040100
}


In [3]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_pageviews_per_article(article_title = None, 
                                  access = None,
                                  endpoint_url = API_REQUEST_PAGEVIEWS_ENDPOINT, 
                                  endpoint_params = API_REQUEST_PER_ARTICLE_PARAMS, 
                                  request_template = ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE,
                                  headers = REQUEST_HEADERS):

    # article title can be as a parameter to the call or in the request_template
    if article_title:
        request_template['article'] = article_title

    if not request_template['article']:
        raise Exception("Must supply an article title to make a pageviews request.")

        
    # article title can be as a parameter to the call or in the request_template
    if access:
        request_template['access'] = access

    if not request_template['access']:
        raise Exception("Must supply an access type to make a pageviews request.")
        
    # Titles are supposed to have spaces replaced with "_" and be URL encoded
    article_title_encoded = urllib.parse.quote(request_template['article'].replace(' ','_'))
    request_template['article'] = article_title_encoded
    
    # now, create a request URL by combining the endpoint_url with the parameters for the request
    request_url = endpoint_url+endpoint_params.format(**request_template)
    
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


In [4]:
# for website data from desktop

for i in range(len(ARTICLE_TITLES)):
    print("Getting pageview data for: ",ARTICLE_TITLES[i])
    views = request_pageviews_per_article(ARTICLE_TITLES[i], 'desktop')
    if 'items' in views.keys():
        print("Have %d months of pageview data"%(len(views['items'])))
        with open("academy_monthly_desktop_start201507-end202310.json", "a") as outfile:
            for month in views['items']:
                outfile.write(json.dumps(month,indent=4))

Getting pageview data for:  Everything Everywhere All at Once
Have 45 months of pageview data
Getting pageview data for:  All Quiet on the Western Front (2022 film)
Have 24 months of pageview data
Getting pageview data for:  The Whale (2022 film)
Have 15 months of pageview data
Getting pageview data for:  Top Gun: Maverick
Have 76 months of pageview data
Getting pageview data for:  Black Panther: Wakanda Forever
Have 29 months of pageview data
Getting pageview data for:  Avatar: The Way of Water
Have 59 months of pageview data
Getting pageview data for:  Women Talking (film)
Have 11 months of pageview data
Getting pageview data for:  Guillermo del Toro's Pinocchio
Have 21 months of pageview data
Getting pageview data for:  Navalny (film)
Have 21 months of pageview data
Getting pageview data for:  The Elephant Whisperers
Have 10 months of pageview data
Getting pageview data for:  An Irish Goodbye
Have 42 months of pageview data
Getting pageview data for:  The Boy, the Mole, the Fox and 

Have 79 months of pageview data
Getting pageview data for:  Suicide Squad (2016 film)
Have 93 months of pageview data
Getting pageview data for:  The White Helmets (film)
Have 81 months of pageview data
Getting pageview data for:  Zootopia
Have 99 months of pageview data
Getting pageview data for:  Spotlight (film)
Have 99 months of pageview data
Getting pageview data for:  Mad Max: Fury Road
Have 99 months of pageview data
Getting pageview data for:  The Revenant (2015 film)
Have 99 months of pageview data
Getting pageview data for:  Bridge of Spies (film)
Have 99 months of pageview data
Getting pageview data for:  The Big Short (film)
Have 99 months of pageview data
Getting pageview data for:  The Danish Girl (film)
Have 99 months of pageview data
Getting pageview data for:  Room (2015 film)
Have 99 months of pageview data
Getting pageview data for:  The Hateful Eight
Have 99 months of pageview data
Getting pageview data for:  Ex Machina (film)
Have 99 months of pageview data
Getting

Getting pageview data for:  Logorama
Have 99 months of pageview data
Getting pageview data for:  Slumdog Millionaire
Have 99 months of pageview data
Getting pageview data for:  The Curious Case of Benjamin Button (film)
Have 99 months of pageview data
Getting pageview data for:  Milk (2008 American film)
Have 40 months of pageview data
Getting pageview data for:  The Dark Knight (film)
Have 99 months of pageview data
Getting pageview data for:  WALL-E
Have 99 months of pageview data
Getting pageview data for:  The Reader (2008 film)
Have 99 months of pageview data
Getting pageview data for:  The Duchess (film)
Have 99 months of pageview data
Getting pageview data for:  Departures (2008 film)
Have 98 months of pageview data
Getting pageview data for:  Vicky Cristina Barcelona
Have 99 months of pageview data
Getting pageview data for:  Smile Pinki
Have 99 months of pageview data
Getting pageview data for:  Man on Wire
Have 99 months of pageview data
Getting pageview data for:  Toyland (f

Have 99 months of pageview data
Getting pageview data for:  Spirited Away
Have 99 months of pageview data
Getting pageview data for:  Nowhere in Africa
Have 99 months of pageview data
Getting pageview data for:  The ChubbChubbs!
Have 99 months of pageview data
Getting pageview data for:  Twin Towers (film)
Have 99 months of pageview data
Getting pageview data for:  Bowling for Columbine
Have 99 months of pageview data
Getting pageview data for:  8 Mile (film)
Have 99 months of pageview data
Getting pageview data for:  A Beautiful Mind (film)
Have 99 months of pageview data
Getting pageview data for:  The Lord of the Rings: The Fellowship of the Ring
Have 99 months of pageview data
Getting pageview data for:  Moulin Rouge!
Have 99 months of pageview data
Getting pageview data for:  Black Hawk Down (film)
Have 99 months of pageview data
Getting pageview data for:  Gosford Park
Have 99 months of pageview data
Getting pageview data for:  Monsters, Inc.
Have 99 months of pageview data
Getti

Have 99 months of pageview data
Getting pageview data for:  One Survivor Remembers
Have 99 months of pageview data
Getting pageview data for:  Antonia's Line
Have 99 months of pageview data
Getting pageview data for:  Toy Story
Have 99 months of pageview data
Getting pageview data for:  Forrest Gump
Have 99 months of pageview data
Getting pageview data for:  The Lion King
Have 99 months of pageview data
Getting pageview data for:  Speed (1994 film)
Have 99 months of pageview data
Getting pageview data for:  Ed Wood (film)
Have 99 months of pageview data
Getting pageview data for:  Pulp Fiction
Have 99 months of pageview data
Getting pageview data for:  Bullets Over Broadway
Have 99 months of pageview data
Getting pageview data for:  The Madness of King George
Have 99 months of pageview data
Getting pageview data for:  Legends of the Fall
Have 99 months of pageview data
Getting pageview data for:  A Time for Justice
Have 99 months of pageview data
Getting pageview data for:  Franz Kafka

...

Have 99 months of pageview data
Getting pageview data for:  Cimarron (1931 film)
Have 99 months of pageview data
Getting pageview data for:  The Circus (1928 film)
Have 52 months of pageview data
Getting pageview data for:  Citizen Kane
Have 99 months of pageview data
Getting pageview data for:  City of Wax
Have 99 months of pageview data
Getting pageview data for:  Cleopatra (1934 film)
Have 99 months of pageview data
Getting pageview data for:  Cleopatra (1963 film)
Have 99 months of pageview data
Getting pageview data for:  Climbing the Matterhorn
Have 99 months of pageview data
Getting pageview data for:  Come and Get It (1936 film)
Have 63 months of pageview data
Getting pageview data for:  Come Back, Little Sheba (1952 film)
Have 99 months of pageview data
Getting pageview data for:  Coquette (film)
Have 99 months of pageview data
Getting pageview data for:  The Country Cousin
Have 99 months of pageview data
Getting pageview data for:  The Country Girl (1954 film)
Have 99 months 

Have 99 months of pageview data
Getting pageview data for:  The Greatest Show on Earth (film)
Have 88 months of pageview data
Getting pageview data for:  Green Dolphin Street (film)
Have 48 months of pageview data
Getting pageview data for:  The Guns of Navarone (film)
Have 99 months of pageview data
Getting pageview data for:  Hamlet (1948 film)
Have 99 months of pageview data
Getting pageview data for:  Heureux Anniversaire
Have 99 months of pageview data
Getting pageview data for:  Harvey (1950 film)
Have 44 months of pageview data
Getting pageview data for:  The Harvey Girls
Have 99 months of pageview data
Getting pageview data for:  Heavenly Music
Have 99 months of pageview data
Getting pageview data for:  The Heiress
Have 99 months of pageview data
Getting pageview data for:  Helen Keller in Her Story
Have 99 months of pageview data
Getting pageview data for:  Hello, Frisco, Hello
Have 99 months of pageview data
Getting pageview data for:  Henry V (1944 film)
Have 99 months of pa

Have 99 months of pageview data
Getting pageview data for:  Mighty Joe Young (1949 film)
Have 99 months of pageview data
Getting pageview data for:  Mildred Pierce (film)
Have 99 months of pageview data
Getting pageview data for:  The Milky Way (1940 film)
Have 99 months of pageview data
Getting pageview data for:  Min and Bill
Have 99 months of pageview data
Getting pageview data for:  Miracle on 34th Street
Have 99 months of pageview data
Getting pageview data for:  The Miracle Worker (1962 film)
Have 99 months of pageview data
Getting pageview data for:  Mister Roberts (1955 film)
Have 99 months of pageview data
Getting pageview data for:  Monsieur Vincent
Have 99 months of pageview data
Getting pageview data for:  Moonbird
Have 99 months of pageview data
Getting pageview data for:  The More the Merrier
Have 99 months of pageview data
Getting pageview data for:  Morning Glory (1933 film)
Have 99 months of pageview data
Getting pageview data for:  Moscow Strikes Back
Have 99 months o

Have 99 months of pageview data
Getting pageview data for:  The Secret Land
Have 99 months of pageview data
Getting pageview data for:  Seeds of Destiny
Have 99 months of pageview data
Getting pageview data for:  Separate Tables (film)
Have 99 months of pageview data
Getting pageview data for:  Serengeti Shall Not Die
Have 99 months of pageview data
Getting pageview data for:  Sergeant York (film)
Have 99 months of pageview data
Getting pageview data for:  Seven Brides for Seven Brothers
Have 99 months of pageview data
Getting pageview data for:  Seven Days to Noon
Have 99 months of pageview data
Getting pageview data for:  The Seventh Veil
Have 99 months of pageview data
Getting pageview data for:  Shane (film)
Have 99 months of pageview data
Getting pageview data for:  Shanghai Express (film)
Have 99 months of pageview data
Getting pageview data for:  She Wore a Yellow Ribbon
Have 99 months of pageview data
Getting pageview data for:  Shoeshine (film)
Have 99 months of pageview data


Have 99 months of pageview data
Getting pageview data for:  Twelve O'Clock High
Have 99 months of pageview data
Getting pageview data for:  Two Arabian Knights
Have 99 months of pageview data
Getting pageview data for:  The Two Mouseketeers
Have 99 months of pageview data
Getting pageview data for:  Two Women
Have 99 months of pageview data
Getting pageview data for:  The Ugly Duckling (1939 film)
Have 99 months of pageview data
Getting pageview data for:  Underworld (1927 film)
Have 99 months of pageview data
Getting pageview data for:  The V.I.P.s (film)
Have 91 months of pageview data
Getting pageview data for:  Perfect Strangers (1945 film)
Have 99 months of pageview data
Getting pageview data for:  Van Gogh (1948 film)
Have 99 months of pageview data
Getting pageview data for:  The Vanishing Prairie
Have 99 months of pageview data
Getting pageview data for:  The Virgin Spring
Have 99 months of pageview data
Getting pageview data for:  Viva Villa!
Have 99 months of pageview data
Ge

In [5]:
# for mobile-app

for i in range(len(ARTICLE_TITLES)):
    print("Getting pageview data for: ",ARTICLE_TITLES[i], 'on mobile-app')
    views = request_pageviews_per_article(ARTICLE_TITLES[i], 'mobile-app')
    if 'items' in views.keys():
        print("Have %d months of pageview data"%(len(views['items'])))
        with open("academy_monthly_mobile_start201507-end202310.json", "a") as outfile:
            for month in views['items']:
                outfile.write(json.dumps(month,indent=4))

Getting pageview data for:  Everything Everywhere All at Once on mobile-app
Have 45 months of pageview data
Getting pageview data for:  All Quiet on the Western Front (2022 film) on mobile-app
Have 24 months of pageview data
Getting pageview data for:  The Whale (2022 film) on mobile-app
Have 15 months of pageview data
Getting pageview data for:  Top Gun: Maverick on mobile-app
Have 76 months of pageview data
Getting pageview data for:  Black Panther: Wakanda Forever on mobile-app
Have 29 months of pageview data
Getting pageview data for:  Avatar: The Way of Water on mobile-app
Have 59 months of pageview data
Getting pageview data for:  Women Talking (film) on mobile-app
Have 11 months of pageview data
Getting pageview data for:  Guillermo del Toro's Pinocchio on mobile-app
Have 21 months of pageview data
Getting pageview data for:  Navalny (film) on mobile-app
Have 21 months of pageview data
Getting pageview data for:  The Elephant Whisperers on mobile-app
Have 10 months of pageview d

Have 69 months of pageview data
Getting pageview data for:  Icarus (2017 film) on mobile-app
Have 74 months of pageview data
Getting pageview data for:  The Silent Child on mobile-app
Have 69 months of pageview data
Getting pageview data for:  Flesh and Sand on mobile-app
Have 78 months of pageview data
Getting pageview data for:  Moonlight (2016 film) on mobile-app
Have 96 months of pageview data
Getting pageview data for:  La La Land (film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Hacksaw Ridge on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Manchester by the Sea (film) on mobile-app
Have 93 months of pageview data
Getting pageview data for:  Arrival (film) on mobile-app
Have 88 months of pageview data
Getting pageview data for:  Fences (film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Fantastic Beasts and Where to Find Them (film) on mobile-app
Have 99 months of pageview data
Getting pageview d

Have 99 months of pageview data
Getting pageview data for:  The Muppets (film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Saving Face (2012 film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Beginners on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Rango (2011 film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  The King's Speech on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Inception on mobile-app
Have 99 months of pageview data
Getting pageview data for:  The Social Network on mobile-app
Have 99 months of pageview data
Getting pageview data for:  The Fighter on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Toy Story 3 on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Alice in Wonderland (2010 film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Black Swan (film) on mobil

Have 99 months of pageview data
Getting pageview data for:  March of the Penguins on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Six Shooter (film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  The Moon and the Son: An Imagined Conversation on mobile-app
Have 99 months of pageview data
Getting pageview data for:  A Note of Triumph: The Golden Age of Norman Corwin on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Tsotsi on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Wallace & Gromit: The Curse of the Were-Rabbit on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Million Dollar Baby on mobile-app
Have 99 months of pageview data
Getting pageview data for:  The Aviator (2004 film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Ray (film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  The Incredibles on mobile-

Have 99 months of pageview data
Getting pageview data for:  The Red Violin on mobile-app
Have 99 months of pageview data
Getting pageview data for:  The Old Man and the Sea (1999 film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  My Mother Dreams the Satan's Disciples in New York on mobile-app
Have 99 months of pageview data
Getting pageview data for:  King Gimp on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Girl, Interrupted (film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  All About My Mother on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Shakespeare in Love on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Saving Private Ryan on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Life Is Beautiful on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Elizabeth (film) on mobile-app
Have 99 months of pageview d

...

Have 99 months of pageview data
Getting pageview data for:  A Double Life (1947 film) on mobile-app
Have 85 months of pageview data
Getting pageview data for:  The Dove (1927 film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Dr. Jekyll and Mr. Hyde (1931 film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Dumbo on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Dylan Thomas (film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  East of Eden (film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Easter Parade (film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Elmer Gantry (film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  The Enemy Below on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Eskimo (1933 film) on mobile-app
Have 12 months of pageview data
Getting pageview data f

Have 99 months of pageview data
Getting pageview data for:  Hud (1963 film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  The Human Comedy (film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  The Hurricane (1937 film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  The Hustler (film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  I Want to Live! on mobile-app
Have 99 months of pageview data
Getting pageview data for:  I Wanted Wings on mobile-app
Have 99 months of pageview data
Getting pageview data for:  I Won't Play on mobile-app
Have 99 months of pageview data
Getting pageview data for:  I'll Cry Tomorrow on mobile-app
Have 99 months of pageview data
Getting pageview data for:  In Beaver Valley on mobile-app
Have 99 months of pageview data
Getting pageview data for:  In Old Arizona on mobile-app
Have 99 months of pageview data
Getting pageview data for:  In Old Chicago on mob

Have 99 months of pageview data
Getting pageview data for:  Moscow Strikes Back on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Mother Wore Tights on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Moulin Rouge (1952 film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Mouse Trouble on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Mr. Deeds Goes to Town on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Mr. Smith Goes to Washington on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Mrs. Miniver on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Munro (film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  The Music Box on mobile-app
Have 99 months of pageview data
Getting pageview data for:  The Music Man (1962 film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Muti

Have 71 months of pageview data
Getting pageview data for:  The Sea Around Us (film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Seal Island (film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  The Search on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Seawards the Great Ships on mobile-app
Have 99 months of pageview data
Getting pageview data for:  The Secret Land on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Seeds of Destiny on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Separate Tables (film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Serengeti Shall Not Die on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Sergeant York (film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Seven Brides for Seven Brothers on mobile-app
Have 99 months of pageview data
Getting pagevie

Have 99 months of pageview data
Getting pageview data for:  The Titan: Story of Michelangelo on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Titanic (1953 film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  To Catch a Thief on mobile-app
Have 99 months of pageview data
Getting pageview data for:  To Each His Own (1946 film) on mobile-app
Have 24 months of pageview data
Getting pageview data for:  To Kill a Mockingbird (film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Tom Jones (1963 film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Tom Thumb (film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Toot, Whistle, Plunk and Boom on mobile-app
Have 99 months of pageview data
Getting pageview data for:  Topkapi (film) on mobile-app
Have 99 months of pageview data
Getting pageview data for:  The Tortoise and the Hare (film) on mobile-app
Have 99 months o

In [6]:
# for mobile-web

for i in range(len(ARTICLE_TITLES)):
    print("Getting pageview data for: ",ARTICLE_TITLES[i], 'on mobile-web')
    views = request_pageviews_per_article(ARTICLE_TITLES[i], 'mobile-web')
    if 'items' in views.keys():
        print("Have %d months of pageview data"%(len(views['items'])))
        with open("academy_monthly_mobile_start201507-end202310.json", "a") as outfile:
            for month in views['items']:
                outfile.write(json.dumps(month,indent=4))

Getting pageview data for:  Everything Everywhere All at Once on mobile-web
Have 45 months of pageview data
Getting pageview data for:  All Quiet on the Western Front (2022 film) on mobile-web
Have 24 months of pageview data
Getting pageview data for:  The Whale (2022 film) on mobile-web
Have 15 months of pageview data
Getting pageview data for:  Top Gun: Maverick on mobile-web
Have 76 months of pageview data
Getting pageview data for:  Black Panther: Wakanda Forever on mobile-web
Have 29 months of pageview data
Getting pageview data for:  Avatar: The Way of Water on mobile-web
Have 59 months of pageview data
Getting pageview data for:  Women Talking (film) on mobile-web
Have 11 months of pageview data
Getting pageview data for:  Guillermo del Toro's Pinocchio on mobile-web
Have 21 months of pageview data
Getting pageview data for:  Navalny (film) on mobile-web
Have 21 months of pageview data
Getting pageview data for:  The Elephant Whisperers on mobile-web
Have 10 months of pageview d

Have 69 months of pageview data
Getting pageview data for:  Icarus (2017 film) on mobile-web
Have 74 months of pageview data
Getting pageview data for:  The Silent Child on mobile-web
Have 69 months of pageview data
Getting pageview data for:  Flesh and Sand on mobile-web
Have 78 months of pageview data
Getting pageview data for:  Moonlight (2016 film) on mobile-web
Have 96 months of pageview data
Getting pageview data for:  La La Land (film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Hacksaw Ridge on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Manchester by the Sea (film) on mobile-web
Have 93 months of pageview data
Getting pageview data for:  Arrival (film) on mobile-web
Have 88 months of pageview data
Getting pageview data for:  Fences (film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Fantastic Beasts and Where to Find Them (film) on mobile-web
Have 99 months of pageview data
Getting pageview d

Have 99 months of pageview data
Getting pageview data for:  The Muppets (film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Saving Face (2012 film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Beginners on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Rango (2011 film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  The King's Speech on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Inception on mobile-web
Have 99 months of pageview data
Getting pageview data for:  The Social Network on mobile-web
Have 99 months of pageview data
Getting pageview data for:  The Fighter on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Toy Story 3 on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Alice in Wonderland (2010 film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Black Swan (film) on mobil

Have 99 months of pageview data
Getting pageview data for:  March of the Penguins on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Six Shooter (film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  The Moon and the Son: An Imagined Conversation on mobile-web
Have 99 months of pageview data
Getting pageview data for:  A Note of Triumph: The Golden Age of Norman Corwin on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Tsotsi on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Wallace & Gromit: The Curse of the Were-Rabbit on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Million Dollar Baby on mobile-web
Have 99 months of pageview data
Getting pageview data for:  The Aviator (2004 film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Ray (film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  The Incredibles on mobile-

Have 99 months of pageview data
Getting pageview data for:  The Red Violin on mobile-web
Have 99 months of pageview data
Getting pageview data for:  The Old Man and the Sea (1999 film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  My Mother Dreams the Satan's Disciples in New York on mobile-web
Have 99 months of pageview data
Getting pageview data for:  King Gimp on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Girl, Interrupted (film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  All About My Mother on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Shakespeare in Love on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Saving Private Ryan on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Life Is Beautiful on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Elizabeth (film) on mobile-web
Have 99 months of pageview d

...

Have 99 months of pageview data
Getting pageview data for:  A Double Life (1947 film) on mobile-web
Have 85 months of pageview data
Getting pageview data for:  The Dove (1927 film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Dr. Jekyll and Mr. Hyde (1931 film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Dumbo on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Dylan Thomas (film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  East of Eden (film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Easter Parade (film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Elmer Gantry (film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  The Enemy Below on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Eskimo (1933 film) on mobile-web
Have 12 months of pageview data
Getting pageview data f

Have 99 months of pageview data
Getting pageview data for:  Hud (1963 film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  The Human Comedy (film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  The Hurricane (1937 film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  The Hustler (film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  I Want to Live! on mobile-web
Have 99 months of pageview data
Getting pageview data for:  I Wanted Wings on mobile-web
Have 99 months of pageview data
Getting pageview data for:  I Won't Play on mobile-web
Have 99 months of pageview data
Getting pageview data for:  I'll Cry Tomorrow on mobile-web
Have 99 months of pageview data
Getting pageview data for:  In Beaver Valley on mobile-web
Have 99 months of pageview data
Getting pageview data for:  In Old Arizona on mobile-web
Have 99 months of pageview data
Getting pageview data for:  In Old Chicago on mob

Have 99 months of pageview data
Getting pageview data for:  Moscow Strikes Back on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Mother Wore Tights on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Moulin Rouge (1952 film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Mouse Trouble on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Mr. Deeds Goes to Town on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Mr. Smith Goes to Washington on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Mrs. Miniver on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Munro (film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  The Music Box on mobile-web
Have 99 months of pageview data
Getting pageview data for:  The Music Man (1962 film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Muti

Have 71 months of pageview data
Getting pageview data for:  The Sea Around Us (film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Seal Island (film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  The Search on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Seawards the Great Ships on mobile-web
Have 99 months of pageview data
Getting pageview data for:  The Secret Land on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Seeds of Destiny on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Separate Tables (film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Serengeti Shall Not Die on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Sergeant York (film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Seven Brides for Seven Brothers on mobile-web
Have 99 months of pageview data
Getting pagevie

Have 99 months of pageview data
Getting pageview data for:  The Titan: Story of Michelangelo on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Titanic (1953 film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  To Catch a Thief on mobile-web
Have 99 months of pageview data
Getting pageview data for:  To Each His Own (1946 film) on mobile-web
Have 24 months of pageview data
Getting pageview data for:  To Kill a Mockingbird (film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Tom Jones (1963 film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Tom Thumb (film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Toot, Whistle, Plunk and Boom on mobile-web
Have 99 months of pageview data
Getting pageview data for:  Topkapi (film) on mobile-web
Have 99 months of pageview data
Getting pageview data for:  The Tortoise and the Hare (film) on mobile-web
Have 99 months o